# Primate Vocalization Detection Pipeline
## Reproducible End-to-End System

This notebook demonstrates a complete pipeline for detecting primate vocalizations in long audio recordings.

### Pipeline Overview:
1. **Configuration** - Set all parameters
2. **Data Loading** - Load species clips and background noise
3. **Preprocessing** - Convert to mel-spectrograms
4. **Data Augmentation** - Apply augmentation strategies
5. **Model Training** - Train VGG19-based classifier
6. **Detection** - Detect vocalizations in long audio files
7. **Visualization** - Create visualizations and reports

### Key Features:
- ✅ Modular design - easy to add new species
- ✅ Configurable parameters - adjust without code changes
- ✅ Reproducible results - fixed random seeds
- ✅ GPU-accelerated - optimized for Google Colab

## 0. Setup & Installation

In [ ]:
# Install required packages
!pip install -q librosa soundfile tensorflow scikit-learn pandas matplotlib

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

print("✅ Setup complete!")

## 1. Configuration

All parameters are defined in `config.py`. To add a new species or change parameters, simply edit the config file.

In [ ]:
# Import all modules
import config
import data_loader
import preprocessing
import augmentation
import model as model_module
import train
import detection
import utils

# Print configuration summary
config.print_config_summary()

### 💡 To Add a New Species:

1. Add audio files to Google Drive: `chimp-audio/audio/new_species_folder/`
2. Edit `config.py` and add to `SPECIES_FOLDERS`:
   ```python
   SPECIES_FOLDERS = {
       'Cercocebus_torquatus': 'Cercocebus torquatus hack 5s',
       'Colobus_guereza': 'Colobus guereza Clips 5s',
       'New_Species': 'new_species_folder',  # ← Add this line
   }
   ```
3. Re-run the notebook - everything will automatically adjust!

## 2. Data Loading & Exploration

Load all audio files and verify the data.

In [ ]:
# Load species data
species_data = data_loader.load_species_data()

# Load background data
background_data = data_loader.load_background_data()

# Print summary
data_loader.print_data_summary(species_data, background_data)

In [ ]:
# Optional: Listen to a sample
import IPython.display as ipd

# Play a Cercocebus sample
if 'Cercocebus_torquatus' in species_data:
    sample_audio, sample_path = species_data['Cercocebus_torquatus'][0]
    print(f"Playing: {sample_path}")
    ipd.display(ipd.Audio(sample_audio, rate=config.SAMPLE_RATE))

## 3. Complete Training Pipeline

This runs the entire training process:
- Data preprocessing
- Augmentation
- Model training
- Evaluation

**This will take some time (~30-60 minutes depending on data size)**

In [ ]:
# Run complete training pipeline
trained_model = train.run_complete_training_pipeline()

## 4. Detection on Long Audio

Now we'll use the trained model to detect primate vocalizations in long audio files.

### 4.1 Test on First Long Audio File

In [ ]:
# Get list of long audio files
long_audio_files = data_loader.get_long_audio_files()

print(f"Found {len(long_audio_files)} long audio files:")
for i, file in enumerate(long_audio_files[:10], 1):  # Show first 10
    print(f"  {i}. {os.path.basename(file)}")
if len(long_audio_files) > 10:
    print(f"  ... and {len(long_audio_files) - 10} more")

In [ ]:
# Detect in the first long audio file
first_audio = long_audio_files[0]
print(f"Processing: {os.path.basename(first_audio)}")

detections_df = detection.detect_in_long_audio(
    trained_model, 
    first_audio,
    confidence_threshold=config.DETECTION_CONFIDENCE_THRESHOLD
)

# Display results
print("\n📋 Detection Results:")
if len(detections_df) > 0:
    display(detections_df.head(20))  # Show first 20 detections
else:
    print("No detections found.")

In [ ]:
# Save detections to CSV
csv_path = detection.save_detections(
    detections_df, 
    os.path.basename(first_audio)
)
print(f"Detections saved to: {csv_path}")

### 4.2 Visualize Detection Results

In [ ]:
# Create visualization
utils.visualize_detection_results(
    first_audio,
    detections_df,
    save_path=None,  # Set to path to save, None to just display
    show_spectrogram=True
)

### 4.3 Process All Long Audio Files

**Warning: This may take a long time if you have many files!**

In [ ]:
# Process all long audio files
# Comment out this cell if you only want to process the first file

all_detections = detection.process_all_long_audio_files(
    trained_model,
    confidence_threshold=config.DETECTION_CONFIDENCE_THRESHOLD
)

## 5. Analysis & Reporting

In [ ]:
# Print detection statistics
utils.print_detection_statistics(all_detections)

In [ ]:
# Create summary report
summary_path = os.path.join(config.DETECTION_OUTPUT_DIR, 'detection_summary.csv')
summary_df = utils.create_detection_summary_report(all_detections, summary_path)

print("\n📊 Summary Report:")
display(summary_df)

In [ ]:
# Create visualizations for all files
utils.visualize_all_detections(all_detections)

## 6. Optional: Extract Detected Clips

Extract audio clips for each detection for manual validation.

In [ ]:
# Extract clips from first audio file
clips_output_dir = os.path.join(config.OUTPUT_ROOT, 'detected_clips')

if len(detections_df) > 0:
    utils.extract_detected_audio_clips(
        first_audio,
        detections_df,
        clips_output_dir,
        padding=0.5  # Add 0.5s padding around each detection
    )
else:
    print("No detections to extract.")

## 7. Model Persistence

The best model is automatically saved during training. You can also manually save/load models.

In [ ]:
# Best model was saved during training
best_model_path = os.path.join(config.MODEL_SAVE_DIR, 'best_model.h5')
print(f"Best model saved at: {best_model_path}")

# To load the model later:
# loaded_model = model_module.load_trained_model(best_model_path)

## 8. Adjust Detection Threshold (Optional)

If you want to experiment with different confidence thresholds without retraining:

In [ ]:
# Try different thresholds
thresholds = [0.5, 0.7, 0.9]

for threshold in thresholds:
    print(f"\n{'='*70}")
    print(f"Testing threshold: {threshold}")
    print(f"{'='*70}")
    
    detections = detection.detect_in_long_audio(
        trained_model,
        first_audio,
        confidence_threshold=threshold
    )
    
    print(f"\nFound {len(detections)} detections with threshold {threshold}")

## Summary

### 📁 Output Files Generated:

```
drive/MyDrive/chimp-audio/outputs/
├── models/
│   ├── best_model.h5              # Trained model
│   ├── training_history.json      # Training metrics
│   └── training_history.png       # Training curves
├── detections/
│   ├── *_detections.csv           # Detection results (CSV)
│   └── detection_summary.csv      # Overall summary
├── visualizations/
│   └── *_visualization.png        # Waveform/spectrogram plots
└── detected_clips/
    └── *.wav                      # Extracted audio clips
```

### 🔄 To Add New Species or Data:

1. Add new audio files to appropriate folders in Google Drive
2. Update `config.py` (add to `SPECIES_FOLDERS` or `BACKGROUND_FOLDERS`)
3. Re-run this notebook

That's it! The pipeline is fully automated and reproducible. 🎉